In [2]:
import torch
from loss import *

In [5]:
# Calculate the expected loss manually (using MSE loss)
mse_loss = nn.MSELoss(reduction="none")
masked_loss = MaskedLoss(mse_loss)


y_hat = torch.tensor(
    [
        [[0.9, 0.8], [0.7, 0.6], [0.5, 0.4], [0.3, 0.2], [0.1, 0.0]],
        [[0.5, 0.6], [0.7, 0.8], [0.9, 1.0], [1.1, 1.2], [1.3, 1.4]],
        [[1.0, 0.9], [0.8, 0.7], [0.6, 0.5], [0.4, 0.3], [0.2, 0.1]],
    ],
    dtype=torch.float32,
)
y = torch.tensor(
    [
        [[1.0, 0.8], [0.7, 0.6], [0.5, 0.4], [0.3, 0.2], [0.1, 0.0]],
        [[0.5, 0.6], [0.7, 0.8], [0.9, 1.0], [1.1, 1.2], [1.3, 1.4]],
        [[1.0, 0.9], [0.8, 0.7], [0.6, 0.5], [0.4, 0.3], [0.2, 0.1]],
    ],
    dtype=torch.float32,
)
mask = torch.tensor(
    [
        [1, 0, 1, 1, 0],
        [1, 1, 0, 1, 0],
        [1, 1, 1, 1, 1],
    ],
    dtype=torch.float32,
)

# Calculate the loss using the MaskedLoss class
masked_loss = masked_loss.get_loss(y_hat, y, mask)
expected_loss = mse_loss(y_hat, y)

In [6]:
masked_loss

tensor(0.0005)

In [8]:
torch.sum(expected_loss)

tensor(0.0100)